# Model training

- ## Data ingestion
- ## Data size,shape, missings (pandas profile)
- ## Feature engineering and histograms
- ## Breakdowns by groups
- ## Split 80 - 20
- ## Model training XGB + pipe with power scaler
- ## Grid search on hyperparameters with CV
- ## Plots and metrics including exporting
- ## Export weights

# Model casting

- ## creating job (days missing)
- ## Data ingestion
- ## feature engineering
- ## calling weights
- ## predictions
- ## appending to table

In [64]:
from datetime import date, datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

import config

engine = create_engine(
    f"mysql+pymysql://{config.sql_credentials['user']}:{config.sql_credentials['password']}@{config.sql_credentials['host']}:{config.sql_credentials['port']}/{config.sql_credentials['db']}",
    pool_pre_ping=True)

conn = engine.connect()

In [ ]:

p=['grouped_days_worked_past_90','grouped_apps_past_7_days',
'','distance_km','']


In [92]:
curdate = date.today()
print(curdate)

placement_query = f'''
SELECT
       placement_ad_id,
       published_at,
       eligible_carer_ids,
       min_provider_rate,
       if( ethnicities_at_risk regexp 'BLACK_ALL',1,0) as ethnicities_at_risk_BLACK_ALL,
       DATEDIFF(published_at,cr.max_date_of_birth)/365.25 as max_cr_age,
       if(recipient_1_gender = 'MALE',1,0) as recipient_1_male,
       if(one_off_payments_total IS NULL,0,one_off_payments_total) as one_off_payments_total,
       if(has_pets regexp 'DOGS',1,0) as has_dogs,
       GREATEST(0,LOG(DATEDIFF(start_date,published_at))) as log_lead_time,
       if(end_date IS NULL,1,0) as ongoing,
       difficulty_normalised,
       if(a.postcode_sector REGEXP'[A-Z][A-Z]',UPPER(substring(a.postcode_sector,1,2)),UPPER(substring(a.postcode_sector,1,1))) as postcode_area_placement,
       SUBSTRING(SUBSTRING(placement_ad_id,6),1,CHAR_LENGTH(SUBSTRING(placement_ad_id,6))-5) as match_request_id
FROM live_STATS_MATCHING_PLACEMENT_AD a
LEFT JOIN (
    SELECT
           mr.match_request_id,
           difficulty_normalised,
           c.customer_id,
           if(c.recipient_2_date_of_birth IS NOT NULL,LEAST(c.recipient_1_date_of_birth, c.recipient_2_date_of_birth),c.recipient_1_date_of_birth) as max_date_of_birth,
           car.gender as recipient_1_gender,
           c.ethnicities_at_risk
    FROM live_STATS_MATCHING_MATCH_REQUEST mr
    LEFT JOIN live_STATS_CUSTOMERS c
    ON mr.account_id = c.customer_id
    LEFT JOIN live_STATS_CUSTOMERS_CARE_APPRAISAL_RECIPIENT car
    ON car.account_id = mr.account_id
    LEFT JOIN live_STATS_MATCHING_DIFFICULTY md
    ON (mr.match_request_id = md.match_request_id)
    ) cr
ON cr.match_request_id = a.match_request_id
WHERE DATE_FORMAT(published_at, '%%Y-%%m-%%d') = '{curdate}'
'''

placement_data = pd.read_sql(placement_query,conn)
placement_data.reset_index(inplace=True)

2021-09-30


In [93]:
placement_data['eligible_carer_ids']=placement_data.eligible_carer_ids.apply(lambda x: x.replace(' ','').split(','))

placement_data=placement_data.explode('eligible_carer_ids').reset_index()
placement_data.drop(['level_0','index'], axis = 1, inplace=True)
placement_data.rename(columns={'eligible_carer_ids':'carer_id'}, inplace = True)

placement_data

,placement_ad_id,published_at,carer_id,min_provider_rate,ethnicities_at_risk_BLACK_ALL,max_cr_age,recipient_1_male,one_off_payments_total,has_dogs,log_lead_time,ongoing,difficulty_normalised,postcode_area_placement,match_request_id
0,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p83637819,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...
1,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p149291474,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...
2,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p220951067,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...
3,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p129992250,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...
4,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p229419193,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54794,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p83066404,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...
54795,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p146299409,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...
54796,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p39241355,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...
54797,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p125708531,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...


In [94]:
carer_query = f'''
SELECT
    professional_id as carer_id,
    UPPER(postcode_area) as postcode_area_carer,
    GREATEST(LOG(DATEDIFF(CURDATE(),p.activated_at)/365.25),0) as log_years_on_elder,
    if(p.gender = 'FEMALE',1,0) as carer_gender_FEMALE,
    COALESCE(days_worked_past_90,0) as days_worked_past_90,
    COALESCE(apps_past_7_days,0) as apps_past_7_days
FROM live_STATS_PROFESSIONAL p
LEFT JOIN(
    SELECT
       v.professional_id as carer_id,
       count(*) as days_worked_past_90
    FROM dates d
    LEFT JOIN (
        SELECT
               date(start_date_time) as work_date,
               professional_id
        FROM live_STATS_NON_CANCELLED_VISITS
        WHERE professional_id IS NOT NULL
            ) v on d.date> v.work_date AND date_add(d.date, INTERVAL -90 day) <= v.work_date
    WHERE d.date = '{curdate}'
    GROUP BY 1
    ) d
ON d.carer_id = p.professional_id
LEFT JOIN (
     SELECT
       v.carer_id,
       count(*) as apps_past_7_days
    FROM dates d
    LEFT JOIN (
        SELECT
               date(valid_from) as work_date,
               carer_id
        FROM live_STATS_MATCHING_PLACEMENT_AD_APPLICATION
        WHERE carer_id IS NOT NULL
            ) v on d.date > v.work_date AND date_add(d.date, INTERVAL -7 day) <= v.work_date
        WHERE d.date = '{curdate}'
        GROUP BY 1
    ) apps
ON apps.carer_id = p.professional_id
WHERE professional_id IN {tuple(placement_data.carer_id.unique().tolist())}
'''

carer_data = pd.read_sql(carer_query,conn)
carer_data

,carer_id,postcode_area_carer,log_years_on_elder,carer_gender_FEMALE,days_worked_past_90,apps_past_7_days
0,p100146905,M,0.971546,1,0,7
1,p100159862,N,0.000000,1,7,0
2,p100267586,OX,0.241455,1,0,0
3,p100322957,EN,0.000000,1,0,0
4,p100481747,CT,1.102483,0,12,4
...,...,...,...,...,...,...
2633,p99426944,RM,0.839937,0,0,0
2634,p99455378,OL,1.382866,1,18,0
2635,p99816556,OL,1.082281,1,0,0
2636,p99929491,CB,0.609676,1,82,0


2638

In [67]:
travel_times = pd.read_csv('travel_times.csv')
travel_times

,from,to,distance_km
0,AB,AB,0.0
1,AB,AL,873.0
2,AB,B,681.0
3,AB,BA,1017.0
4,AB,BB,534.0
...,...,...,...
14636,ZE,WR,1080.0
14637,ZE,WS,1027.0
14638,ZE,WV,1020.0
14639,ZE,YO,899.0


In [98]:
print(placement_data.shape)
df = placement_data.merge(carer_data, on = 'carer_id', how='left')
print(df.shape)
df = df.merge(travel_times, left_on = ['postcode_area_carer','postcode_area_placement'], right_on=['from','to'], how='left')
df.drop(['from','to'], axis = 1, inplace = True)
print(df.shape)

(54799, 14)
(54799, 19)
(54799, 20)


In [99]:
# keep non-missing
f_predictors = [
'difficulty_normalised','grouped_days_worked_past_90','grouped_apps_past_7_days',
'carer_gender_FEMALE','log_lead_time','distance_km',
'has_dogs','log_years_on_elder', 'max_cr_age',
 'min_provider_rate','ethnicities_at_risk_BLACK_ALL', 'ongoing', 'recipient_1_male',
    'one_off_payments_total',

]

,placement_ad_id,published_at,carer_id,min_provider_rate,ethnicities_at_risk_BLACK_ALL,max_cr_age,recipient_1_male,one_off_payments_total,has_dogs,log_lead_time,ongoing,difficulty_normalised,postcode_area_placement,match_request_id,postcode_area_carer,log_years_on_elder,carer_gender_FEMALE,days_worked_past_90,apps_past_7_days,distance_km
0,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p83637819,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...,NP,0.305994,1,0,0,0.0
1,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p149291474,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...,NP,0.000000,1,8,0,0.0
2,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p220951067,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...,BS,1.110632,1,16,1,42.0
3,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p129992250,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...,BS,0.000000,1,0,4,42.0
4,plad_pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8...,2021-09-30 08:42:28,p229419193,700.0,0,92.4079,1,0.0,0,3.465736,1,1.000000,NP,pl_s00e2ZDNPZqM_4m3d-b7676ec4-c0c4-4a8d-8c6b-5...,BS,0.000000,1,20,0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54794,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p83066404,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...,None,1.003165,1,0,0,NaN
54795,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p146299409,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...,None,0.160875,1,0,8,NaN
54796,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p39241355,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...,None,0.580995,1,45,0,NaN
54797,plad_pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b...,2021-09-30 11:31:03,p125708531,700.0,0,83.5729,0,0.0,0,4.189655,0,0.693629,GU,pl_s0YNf7dGMhMt_RZZG-083e9c2e-2438-4379-b456-f...,None,0.425567,1,0,0,NaN


In [ ]:
def 